# Chapter.9.03 SVLoR for One Sample with Modularization

PA 9.02에서 했던 SVLoR에서<br>
Dataset을 생성할 때 다양한 조건들을 변경하면서<br>
결과값을 비교합니다

### Step.1
PA 9.02에서 사용한 모듈들을 가져옵니다

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import basic_nodes as nodes

class Affine:
    def __init__(self):
        self._feature_dim = 1
        self._Th = None
        
        self.node_imp()
        self.random_initialization()

    def node_imp(self):
        self._node1 = nodes.mul_node()
        self._node2 = nodes.plus_node()
    
    def random_initialization(self):
        r_feature_dim = 1/np.sqrt(self._feature_dim)
        self._Th = np.random.uniform(low = -1 * r_feature_dim,
                                     high = r_feature_dim,
                                     size = (self._feature_dim + 1,1))
    
    def forward(self,x):
        self._z1 = self._node1.forward(self._Th[1], x)
        self._z2 = self._node2.forward(self._Th[0], self._z1)

        return self._z2
    
    def backward(self, dz, lr):
        dth0, dz1 = self._node2.backward(dz)
        dth1, dx = self._node1.backward(dz1)

        self._Th[1] = self._Th[1] - lr*dth1
        self._Th[0] = self._Th[0] - lr*dth0
    
    def get_Th(self):
        return self._Th 

class Sigmoid:
    def __init__(self):
        self._pred = None
    
    def forward(self, z):
        self._pred = 1/(1 + np.exp(-1*z))
        return self._pred
    
    def backward(self, dpred):
        partial = self._pred * (1 - self._pred)
        dz = dpred * partial
        return dz

class BinaryCrossEntropy_Loss:
    def __init__(self):
        self._y, self._pred = None, None
    
    def forward(self, y, pred):
        self._y, self._pred = y, pred
        loss = -1*(y*np.log(self._pred) + (1-y)*np.log(1-pred))
        return loss
    
    def backward(self):
        dpred = (self._pred - self._y)/(self._pred*(1-self._pred))
        return dpred

class SVLoR:
    def __init__(self):
        self._feature_dim = 1
        self._affine = Affine()
        self._sigmoid = Sigmoid()

    def forward(self, x):
        z = self._affine.forward(x)
        pred =  self._sigmoid.forward(z)
        return pred
    
    def backward(self, dpred, lr):
        dz = self._sigmoid.backward(dpred)
        self._affine.backward(dz, lr)
        
    def  get_Th(self):
        return self._affine.get_Th()

def result_tracker():
    global iter_idx, check_freq
    global th_accum, model

    if iter_idx % check_freq == 0:
        th_accum = np.hstack((th_accum, model.get_Th()))
        loss_list.append(loss)
    iter_idx += 1

def result_visualizer():
    global th_accum, loss_list
    fig,ax = plt.subplots(2, 1, figsize = (30,10))
    fig.subplots_adjust(hspace = 0.3)
    ax[0].set_title(r'$\vec{\theta}$' + 'Update ')

    ax[0].plot(th_accum[1,:], label = r'$\theta_{1}$')
    ax[0].plot(th_accum[0,:], label = r'$\theta_{0}$')

    ax[0].legend()
    iter_ticks = np.linspace(0,th_accum.shape[1],10).astype(np.int)
    ax[0].set_xticks(iter_ticks)

    ax[1].set_title(r'$\mathcal{L}$')
    ax[1].plot(loss_list)
    ax[1].set_xticks(iter_ticks)

    n_pred = 1000
    fig,ax = plt.subplots(figsize = (30,10))
    ax.set_title('Predictor Update')
    ax.scatter(data[:,1], data[:,-1])

    ax_idx_arr = np.linspace(0,len(loss_list)-1,n_pred).astype(np.int)
    cmap = cm.get_cmap('rainbow',lut = len(ax_idx_arr))

    x_pred = np.linspace(np.min(data[:,1]),np.max(data[:,1]),1000)
    for ax_cnt, ax_idx in enumerate(ax_idx_arr):
        z = th_accum[1, ax_idx] * x_pred + th_accum[0,ax_idx]
        a = 1/(1 + np.exp(-1 * z))
        ax.plot(x_pred, a, color = cmap(ax_cnt),alpha = 0.2)

    y_ticks = np.round(np.linspace(0, 1, 7),2)
    ax.set_yticks(y_ticks)
def dataset_generator(x_dict):
    x_data = np.random.normal(x_dict['mean'], x_dict['std'],x_dict['n_sample'])
    x_data_noise = x_data + x_dict['noise_factor'] * np.random.normal(0,1,x_dict['n_sample'])
    
    if x_dict['direction'] > 0:
        y_data = (x_data_noise > x_dict['cutoff']).astype(np.int)
    else:
        y_data = (x_data_noise < x_dict['cutoff']).astype(np.int)
    
    data = np.zeros(shape = (x_dict['n_sample'],1))
    data = np.hstack((data,x_data.reshape(-1,1),y_data.reshape(-1,1)))
    return data


### Step.2
비교를 위하여 PA 9.02에서 했던 값을 동일하게 진행합니다<br>
"""<br>
mean: 1<br>
std: 1<br>
n_sample: 300<br>
noise_factor: 0.3<br>
cutoff: 1<br>
direction: -1<br>
iter_idx: 0<br>
check_freq: 5<br>
epochs: 300<br>
learning rate: 0.01<br>
"""<br><br>

In [ ]:
### Start
x_dict =  {'mean': , 'std': , 'n_sample': ,
            'noise_factor': ,
            'cutoff': , 'direction': }

data = 

model = 
BCE_loss = 

th_accum = 

loss_list = []
iter_idx, check_freq = 
epochs, lr =  

for epoch in range(epochs):
    np.random.shuffle(data)
    
    for data_idx in range(data.shape[0]):
        x,y =  

        pred =
        loss = 

        dpred = 
        model.backward(dpred, lr)
### End
        result_tracker()
result_visualizer()

**Expected Output** 
<img src='./img_9/9_05.png' width = 700>
<img src='./img_9/9_06.png' width = 700>

### Step.3
mean과 cutoff를 10으로 증가시키고 실행하여<br>
나머지 조건을 동일하게 하고 결과값을 확인합니다

In [ ]:
### Start
x_dict =  {'mean': , 'std': , 'n_sample': ,
            'noise_factor': ,
            'cutoff': , 'direction': }

data = 

model = 
BCE_loss = 

th_accum =

loss_list = []
iter_idx, check_freq = 
epochs, lr =  

for epoch in range(epochs):
    np.random.shuffle(data)
    
    for data_idx in range(data.shape[0]):
        x,y =

        pred = 
        loss =

        dpred = 
        model.backward(dpred, lr)
### End
        result_tracker()
result_visualizer()

**Expected Output** 
<img src='./img_9/9_07.png' width = 700>
<img src='./img_9/9_08.png' width = 700>

### Step.4
Step.3에서는 data의 mean값이 증가하여서 x1와 x0가 동일하게 학습이 되지 않는 모습을<br>
확인할 수 있는데 따라서 동일 epochs에서 명확한 solution을 얻기 힘듭니다<br>
따라서 epochs를 늘려서 학습하면 어떤 결과가 나오는지 확인합니다

In [ ]:
### Start
x_dict =  {'mean': , 'std': , 'n_sample': ,
            'noise_factor': ,
            'cutoff': , 'direction': }

data = 

model = 
BCE_loss = 

th_accum = 

loss_list = []
iter_idx, check_freq = 
epochs, lr = 

for epoch in range(epochs):
    np.random.shuffle(data)
    
    for data_idx in range(data.shape[0]):
        x,y =  

        pred = 
        loss = 

        dpred =
        model.backward(dpred, lr)
### End
        result_tracker()
result_visualizer()

**Expected Output** 
<img src='./img_9/9_09.png' width = 700>
<img src='./img_9/9_10.png' width = 700>

### Step.5
Step.3, 4에서는 mean값을 증가시켜 학습결과를 관찰했습니다<br>
Step.5에서는 mean값은 초기와 같이 1로 설정하고 std를 변경시켜<br>
학습 결과를 확인할 것입니다<br>
mean:1<br>
std:7<br>
cutoff:1<br>
epochs:1000<br>
learning rate:0.0001

In [ ]:
### Start
x_dict =  {'mean': , 'std': , 'n_sample': ,
            'noise_factor': ,
            'cutoff': , 'direction': }

data = 

model = 
BCE_loss =

th_accum = 

loss_list = []
iter_idx, check_freq = 
epochs, lr = 

for epoch in range(epochs):
    np.random.shuffle(data)
    
    for data_idx in range(data.shape[0]):
        x,y = 

        pred = 
        loss = 

        dpred = 
        model.backward(dpred, lr)
### End
        result_tracker()
result_visualizer()

**Expected Output** 
<img src='./img_9/9_11.png' width = 700>
<img src='./img_9/9_12.png' width = 700>

Std를 증가시키게 되면<br>
mean에서와 비슷하게 x1와 x0의 학습이 불균형하게 일어나는 모습을 볼수 있다